In [1]:
import torch

In [2]:

rope_type = "default"
dim = 64
base = 10000
max_patches_per_side = 1024 // 16
freqs = 1.0 / (base ** (torch.arange(0, dim, 2).float() / dim))


In [4]:
freqs

tensor([1.0000e+00, 7.4989e-01, 5.6234e-01, 4.2170e-01, 3.1623e-01, 2.3714e-01,
        1.7783e-01, 1.3335e-01, 1.0000e-01, 7.4989e-02, 5.6234e-02, 4.2170e-02,
        3.1623e-02, 2.3714e-02, 1.7783e-02, 1.3335e-02, 1.0000e-02, 7.4989e-03,
        5.6234e-03, 4.2170e-03, 3.1623e-03, 2.3714e-03, 1.7783e-03, 1.3335e-03,
        1.0000e-03, 7.4989e-04, 5.6234e-04, 4.2170e-04, 3.1623e-04, 2.3714e-04,
        1.7783e-04, 1.3335e-04])

In [5]:

h = torch.arange(max_patches_per_side, device=freqs.device)
w = torch.arange(max_patches_per_side, device=freqs.device)

In [6]:
h

tensor([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
        18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35,
        36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53,
        54, 55, 56, 57, 58, 59, 60, 61, 62, 63])

In [7]:
freqs_h = torch.outer(h, freqs[::2]).float()
freqs_w = torch.outer(w, freqs[1::2]).float()

In [13]:
freqs_h.shape, freqs_w.shape

(torch.Size([64, 16]), torch.Size([64, 16]))

In [15]:
freqs_h[:, None, :].repeat(1, max_patches_per_side, 1).shape, freqs_w[None, :, :].repeat(max_patches_per_side, 1, 1).shape

(torch.Size([64, 64, 16]), torch.Size([64, 64, 16]))

In [16]:
inv_freq_unreshaped = torch.cat(
    [
        freqs_h[:, None, :].repeat(1, max_patches_per_side, 1),
        freqs_w[None, :, :].repeat(max_patches_per_side, 1, 1),
    ],
    dim=-1,
)
inv_freq_unreshaped.shape

torch.Size([64, 64, 32])

In [17]:
inv_freq = torch.cat(
    [
        freqs_h[:, None, :].repeat(1, max_patches_per_side, 1),
        freqs_w[None, :, :].repeat(max_patches_per_side, 1, 1),
    ],
    dim=-1,
).reshape(-1, dim // 2)  # we reshape to only index on the position indexes, not tuple of indexes
inv_freq.shape

# # TODO maybe make it torch compatible later on. We can also just slice
# self.register_buffer("inv_freq", torch.cat((inv_freq, inv_freq), dim=-1), persistent=False)


torch.Size([4096, 32])